In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
def compute_det_curve(target_scores, nontarget_scores):

    n_scores = target_scores.size + nontarget_scores.size
    all_scores = np.concatenate((target_scores, nontarget_scores))
    labels = np.concatenate((np.ones(target_scores.size), np.zeros(nontarget_scores.size)))

    # Sort labels based on scores
    indices = np.argsort(all_scores, kind='mergesort')
    labels = labels[indices]

    # Compute false rejection and false acceptance rates
    tar_trial_sums = np.cumsum(labels)
    nontarget_trial_sums = nontarget_scores.size - (np.arange(1, n_scores + 1) - tar_trial_sums)

    frr = np.concatenate((np.atleast_1d(0), tar_trial_sums / target_scores.size))  # false rejection rates
    far = np.concatenate((np.atleast_1d(1), nontarget_trial_sums / nontarget_scores.size))  # false acceptance rates
    thresholds = np.concatenate((np.atleast_1d(all_scores[indices[0]] - 0.001), all_scores[indices]))  # Thresholds are the sorted scores

    return frr, far, thresholds


def compute_eer(target_scores, nontarget_scores):
    """ Returns equal error rate (EER) and the corresponding threshold. """
    frr, far, thresholds = compute_det_curve(target_scores, nontarget_scores)
    abs_diffs = np.abs(frr - far)
    min_index = np.argmin(abs_diffs)
    eer = np.mean((frr[min_index], far[min_index]))
    return eer, thresholds[min_index]

def leer_columnas(archivo_csv, c_names, c_labels, c_phase, phase):
    try:
        df = pd.read_csv(archivo_csv, sep=" ", header=None)
        if phase:
            df = df[[c_names, c_labels, c_phase]]
            df.columns = ["ID", "Label", "Phase"]
        else:
            df = df[[c_names, c_labels]]
            df.columns = ["ID", "Label"]

        return df
    except FileNotFoundError:
        print("El archivo de metadatos no existe")
    except KeyError:
        print("Las columnas especificadas no existen en el archivo de metadatos")




In [4]:
def merge(score_file, metadata='Scores/LA/trial_metadata.txt', col_names=1, col_labels=5, phase='eval', col_phase=7):
    metadatos = leer_columnas(metadata, col_names, col_labels, col_phase, phase)
    scores = pd.read_csv(score_file, sep=" ", header=None, skipinitialspace=True)
    scores.columns = ["ID", "Scores"]
    if len(scores) != len(metadatos):
        print("CHECK: submission has %d of %d expected trials." % (len(scores), len(metadatos)))
    if len(scores.columns) > 2:
        print("CHECK: submission has more columns (%d) than expected (2). Check for leading/ending blank spaces."
              % len(scores.columns)
              )
        exit(1)
    if phase:
        cm_scores = scores.merge(metadatos[metadatos["Phase"] == phase], on="ID")
    else:
        cm_scores = scores.merge(metadatos, on="ID")
    return cm_scores

def extract_eer(score_file, metadata='Scores/LA/trial_metadata.txt', col_names=1, col_labels=5, phase='progress', col_phase=7, pri=True):
    cm_scores= merge(score_file, metadata, col_names, col_labels, phase, col_phase)
    bona_cm = cm_scores[cm_scores["Label"] == "bonafide"]["Scores"].values
    spoof_cm = cm_scores[cm_scores["Label"] == "spoof"]["Scores"].values
    eer_cm, threshold = compute_eer(bona_cm, spoof_cm)
    out_data = "EER: {:.4f}. Threshold: {:.5f}\n".format(100 * eer_cm, threshold)
    if pri: print(out_data)
    return eer_cm, threshold


## Analysis Anti-spoofing models

In [18]:
Models=['CQCC_GMM', 'LFCC_GMM', 'LFCC_LCNN', 'RawNet2']
thresholds=[]
eers=[]
Database = 'ASVspoof2019_LA_dev'
for model in Models:
    print(model)
    score_path=os.path.join('Scores/LA', Database, model + '.txt')
    eer, th = extract_eer(score_file = score_path, metadata=os.path.join('Metadata', Database, 'trial_metadata.txt'), col_labels=4, phase=None)
    thresholds.append(th)
    eers.append(eer)


CQCC_GMM
EER: 11.3095. Threshold: 0.73193

LFCC_GMM
EER: 7.2918. Threshold: 0.80453

LFCC_LCNN
CHECK: submission has 24986 of 24844 expected trials.
EER: 0.5506. Threshold: -5.17534

RawNet2
EER: 1.2957. Threshold: -7.05358



In [19]:
Models=['CQCC_GMM', 'LFCC_GMM', 'LFCC_LCNN', 'RawNet2']
thresholds=[]
eers=[]
Database = 'ASVspoof2021_LA_eval'
for model in Models:
    print(model)
    score_path=os.path.join('Scores/LA', Database, model + '.txt')
    eer, th = extract_eer(score_file = score_path, metadata=os.path.join('Metadata', Database, 'trial_metadata.txt'), col_labels=5, phase='eval')
    thresholds.append(th)
    eers.append(eer)


CQCC_GMM
EER: 15.6181. Threshold: 0.79618

LFCC_GMM
EER: 19.2967. Threshold: 1.15404

LFCC_LCNN
EER: 9.2601. Threshold: 5.26733

RawNet2
EER: 7.4716. Threshold: -5.78430



Now with thresholds, we can see which bonafides are classified as spoof and vice versa.

In [20]:
dfs_fallos = []
Database = 'ASVspoof2021_LA_eval'
for i in range(0, len(Models)):
    score_path=os.path.join('Scores/LA', Database, Models[i] + '.txt')
    df_merged = merge(score_path, metadata='Metadata/' + Database + '/trial_metadata.txt', col_names=1, col_labels=5, phase='eval', col_phase=7)
    df_merged

    condition_1 = (df_merged['Label'] == 'bonafide') & (df_merged['Scores'] < thresholds[i])
    condition_2 = (df_merged['Label'] == 'spoof') & (df_merged['Scores'] > thresholds[i])

    selected_rows = df_merged[condition_1 | condition_2]
    dfs_fallos.append(selected_rows)

In [23]:
df = pd.DataFrame()

# Bucles for anidados para filas y columnas
for i in range(len(Models)):  # Bucle externo para filas
    df_fila = dfs_fallos[i]
    for j in range(len(Models)):  # Bucle interno para columnas
        df_columna = dfs_fallos[j]
        # Calcular el número que deseas añadir al DataFrame
        fallos_comun = df_fila.merge(df_columna, on="ID")
        
        # Añadir el número al DataFrame
        df.at[i, j] = len(fallos_comun)

# Mostrar el DataFrame resultante
df = df.astype(int)
df.rename(index=dict(zip(df.index, Models)), columns=dict(zip(df.index, Models)), inplace=True)
print(df)

           CQCC_GMM  LFCC_GMM  LFCC_LCNN  RawNet2
CQCC_GMM      23142     11231       7340     4314
LFCC_GMM      11231     28593       3688     2501
LFCC_LCNN      7340      3688      13721     4678
RawNet2        4314      2501       4678    11071


In [22]:
fallan_todos = dfs_fallos[0]["ID"]
for i in range(len(Models)):  # Bucle externo para filas
    df_fila = dfs_fallos[i]

    fallan_todos = df_fila.merge(fallan_todos, on="ID")["ID"]
        
len(fallan_todos)

1107

## Scores normalization for classic ensemble model


This code was used to generate the normalized scores referenced in the paper. While it is included for transparency, running it is not required, as both the raw and normalized scores are available in the 'Scores' folder of this repository.

In [2]:
import pandas as pd
import numpy as np
import os
import math

Models=['CQCC_GMM', 'LFCC_GMM', 'LFCC_LCNN', 'RawNet2']
Models=['ModeloClasificacion']
def sigmoid_transform(score):
    return 1 / (1 + np.exp(-score))
def min_max_scaling(score, min_val, max_val):
    return (score - min_val) / (max_val - min_val)

for i in range(len(Models)):
    score_file=os.path.join('Scores/LA', 'ASVspoof2021_LA_eval', Models[i] + '.txt') # Change the Name of the dataset to normalize other datasets scores
    scores = pd.read_csv(score_file, sep=" ", header=None, skipinitialspace=True)
    min_score = scores[1].min()
    max_score = scores[1].max()
    scores[1] = scores[1].apply(lambda x: min_max_scaling(x, min_score, max_score))
    scores[1] = scores[1].apply(sigmoid_transform)
    score_file_norm=os.path.join('Scores/LA', 'ASVspoof2021_LA_eval', Models[i] + '_norm.txt') # Change the Name of the dataset to normalize other datasets scores
    scores.to_csv(path_or_buf=score_file_norm, sep=' ', header=False, index=False)



## Error Based (EB) ensemble

In [40]:
Models=['CQCC_GMM', 'LFCC_GMM', 'LFCC_LCNN', 'RawNet2']
thresholds_norm=[]
eers_norm=[]
Database = 'ASVspoof2021_DF_eval'
print('#Normal#')
for model in Models:
    print(model)
    score_path=os.path.join('Scores/DF', Database, model + '_norm.txt')
    eer, th = extract_eer(score_file = score_path, metadata=os.path.join('Metadata', Database, 'trial_metadata.txt'), col_labels=5, phase='progress')
    thresholds_norm.append(th)
    eers_norm.append(eer)

#Normal#
CQCC_GMM
EER: 17.6317. Threshold: 0.70615

LFCC_GMM
EER: 21.0128. Threshold: 0.72422

LFCC_LCNN
EER: 11.6129. Threshold: 0.70594

RawNet2
EER: 5.7913. Threshold: 0.73072



In [47]:
def weighted(score, weight):
    return (score) * weight

Database='ASVspoof2021_DF_eval'
print('#Modelo Fusion Clasico#')
score_path=os.path.join('Scores/DF', Database, Models[0] + '_norm.txt')
print(Models[0])
scores = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
scores[1] = scores[1].apply(lambda x: weighted(x, eers_norm[0]))
modelo_fusion=scores

for i,model in enumerate(Models[1:4]):
    print(model)
    score_path=os.path.join('Scores/DF', Database, model + '_norm.txt')
    scores_new = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    modelo_fusion[1]=modelo_fusion[1] + scores_new[1].apply(lambda x: weighted(x, eers_norm[i]))

#score_file_mf = os.path.join('Scores/DF', Database, 'EB_Ensemble.txt')
#modelo_fusion.to_csv(path_or_buf=score_file_mf, sep=' ', header=False, index=False)



#Modelo Fusion Clasico#
CQCC_GMM
LFCC_GMM
LFCC_LCNN
RawNet2


In [50]:
score_file_mf = os.path.join('Scores/LA', 'ASVspoof2021_LA_eval','EB_Ensemble.txt')
eer, th = extract_eer(score_file = score_file_mf, metadata=os.path.join('Metadata', 'ASVspoof2021_LA_eval', 'trial_metadata.txt'), col_labels=5, phase='eval')

score_file_mf = os.path.join('Scores/DF', 'ASVspoof2021_DF_eval','EB_Ensemble.txt')
eer, th = extract_eer(score_file = score_file_mf, metadata=os.path.join('Metadata', 'ASVspoof2021_DF_eval', 'trial_metadata.txt'), col_labels=5, phase='eval')

EER: 6.6414. Threshold: 0.47428

EER: 20.1504. Threshold: 0.45982



### 4 baseline models + clasification model

In [11]:
Models=['CQCC_GMM', 'LFCC_GMM', 'LFCC_LCNN', 'RawNet2', 'ModeloClasificacion']
thresholds_norm=[]
eers_norm=[]
Database = 'ASVspoof2021_DF_eval'
print('#Normal#')
for model in Models:
    print(model)
    score_path=os.path.join('Scores/DF', Database, model + '_norm.txt')
    eer, th = extract_eer(score_file = score_path, metadata=os.path.join('Metadata', Database, 'trial_metadata.txt'), col_labels=5, phase='progress')
    thresholds_norm.append(th)
    eers_norm.append(eer)

#Normal#
CQCC_GMM
EER: 17.6317. Threshold: 0.70615

LFCC_GMM
EER: 21.0128. Threshold: 0.72422

LFCC_LCNN
EER: 11.6129. Threshold: 0.70594

RawNet2
EER: 5.7913. Threshold: 0.73072

ModeloClasificacion
EER: 3.3801. Threshold: 0.58629



In [53]:
def weighted(score, weight):
    return (score) * weight

Database='ASVspoof2021_DF_eval'
score_path=os.path.join('Scores/DF', Database, Models[0] + '_norm.txt')
print(Models[0])
scores = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
scores[1] = scores[1].apply(lambda x: weighted(x, eers_norm[0]))
modelo_fusion=scores

for i,model in enumerate(Models[1:5]):
    print(model)
    score_path=os.path.join('Scores/DF', Database, model + '_norm.txt')
    scores_new = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    modelo_fusion[1]=modelo_fusion[1] + scores_new[1].apply(lambda x: weighted(x, eers_norm[i]))

#score_file_mf = os.path.join('Scores/DF', Database, 'EB_Ensemble_5_models.txt')
#modelo_fusion.to_csv(path_or_buf=score_file_mf, sep=' ', header=False, index=False)



CQCC_GMM
LFCC_GMM
LFCC_LCNN
RawNet2


In [55]:
score_file_mf = os.path.join('Scores/LA', 'ASVspoof2021_LA_eval','EB_Ensemble_5_models.txt')
eer, th = extract_eer(score_file = score_file_mf, metadata=os.path.join('Metadata', 'ASVspoof2021_LA_eval', 'trial_metadata.txt'), col_labels=5, phase='eval')

score_file_mf = os.path.join('Scores/DF', 'ASVspoof2021_DF_eval','EB_Ensemble_5_models.txt')
eer, th = extract_eer(score_file = score_file_mf, metadata=os.path.join('Metadata', 'ASVspoof2021_DF_eval', 'trial_metadata.txt'), col_labels=5, phase='eval')

EER: 5.7566. Threshold: 0.46809

EER: 12.9785. Threshold: 0.49408



## GS Ensemble

### LA

In [82]:
'''
Models=['CQCC_GMM', 'LFCC_GMM', 'LFCC_LCNN', 'RawNet2']
import itertools
pesos = [0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.7]

# Generar todas las combinaciones posibles de 4 pesos que sumen 1
combinaciones = itertools.product(pesos, repeat=4)

# Filtrar las combinaciones que sumen 1
combinaciones_validas = filter(lambda x: sum(x) == 1, combinaciones)
# Iterar sobre las combinaciones válidas
Database = 'ASVspoof2021_LA_eval'
score_path=os.path.join('Scores/LA', Database, Models[0] + '_norm.txt')
scores = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
modelo_fusion=scores
mejor_err=100
for combinacion in combinaciones_validas:
    score_path=os.path.join('Scores/LA', Database, Models[0] + '_norm.txt')
    scores0 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    score_path=os.path.join('Scores/LA', Database, Models[1] + '_norm.txt')
    scores1 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    score_path=os.path.join('Scores/LA', Database, Models[2] + '_norm.txt')
    scores2 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    score_path=os.path.join('Scores/LA', Database, Models[3] + '_norm.txt')
    scores3 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    p1,p2,p3,p4=combinacion
    modelo_fusion[1] = scores0[1].apply(
        lambda x: weighted(x, p1)) + scores1[1].apply(
        lambda x: weighted(x, p2)) + scores2[1].apply(
        lambda x: weighted(x, p3)) + scores3[1].apply(
        lambda x: weighted(x, p4))
    score_file_mf = os.path.join('Scores/LA', Database, '_Modelo_Fusion.txt')
    modelo_fusion.to_csv(path_or_buf=score_file_mf, sep=' ', header=False, index=False)
    eer, th = extract_eer(score_file = score_file_mf, metadata=os.path.join('Metadata', Database, 'trial_metadata.txt'), col_labels=5, phase='progress', pri=False)
    if eer<mejor_err:
        mejor_err=eer
        score_file_mff = os.path.join('Scores/LA', Database, 'GS_Ensemble.txt')
        modelo_fusion.to_csv(path_or_buf=score_file_mff, sep=' ', header=False, index=False)
        mejor_combinacion=combinacion
        print('Nueva mejor combinación:')
        print(combinacion)
        print('Mejor EER de progress: {}'.format(eer) )
    
'''

Nueva mejor combinación:
(0.1, 0.1, 0.1, 0.7)
Mejor EER de progress: 0.06979769121967563
Nueva mejor combinación:
(0.1, 0.1, 0.2, 0.6)
Mejor EER de progress: 0.06916124988137837
Nueva mejor combinación:
(0.1, 0.1, 0.25, 0.55)
Mejor EER de progress: 0.06869386453442544
Nueva mejor combinación:
(0.1, 0.1, 0.3, 0.5)
Mejor EER de progress: 0.0686262421378877
Nueva mejor combinación:
(0.1, 0.1, 0.35, 0.45)
Mejor EER de progress: 0.06812504559266591
Nueva mejor combinación:
(0.1, 0.1, 0.4, 0.4)
Mejor EER de progress: 0.06631715517258074
Nueva mejor combinación:
(0.1, 0.1, 0.45, 0.35)
Mejor EER de progress: 0.06504427249598624
Nueva mejor combinación:
(0.1, 0.1, 0.5, 0.3)
Mejor EER de progress: 0.06327019327416991
Nueva mejor combinación:
(0.1, 0.1, 0.55, 0.25)
Mejor EER de progress: 0.06018942017749023
Nueva mejor combinación:
(0.1, 0.1, 0.6, 0.2)
Mejor EER de progress: 0.057176269477348274
Nueva mejor combinación:
(0.1, 0.2, 0.6, 0.1)
Mejor EER de progress: 0.0501137637883519
Nueva mejor co

In [58]:
Database = 'ASVspoof2021_LA_eval'
score_file_mff = os.path.join('Scores/LA', Database, 'GS_Ensemble.txt')
eer, th = extract_eer(score_file = score_file_mff, metadata=os.path.join('Metadata', Database, 'trial_metadata.txt'), col_labels=5, phase='eval')

EER: 5.7843. Threshold: 0.70215



#### 4 baseline models + clasification model

In [14]:
'''
Models=['CQCC_GMM', 'LFCC_GMM', 'LFCC_LCNN', 'RawNet2', 'ModeloClasificacion']
import itertools
pesos = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]

# Generar todas las combinaciones posibles de 4 pesos que sumen 1
combinaciones = itertools.product(pesos, repeat=5)

# Filtrar las combinaciones que sumen 1
combinaciones_validas = filter(lambda x: sum(x) == 1, combinaciones)
# Iterar sobre las combinaciones válidas
Database = 'ASVspoof2021_LA_eval'
score_path=os.path.join('Scores/LA', Database, Models[0] + '_norm.txt')
scores = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
modelo_fusion=scores
mejor_err=100
for combinacion in combinaciones_validas:
    score_path=os.path.join('Scores/LA', Database, Models[0] + '_norm.txt')
    scores0 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    score_path=os.path.join('Scores/LA', Database, Models[1] + '_norm.txt')
    scores1 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    score_path=os.path.join('Scores/LA', Database, Models[2] + '_norm.txt')
    scores2 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    score_path=os.path.join('Scores/LA', Database, Models[3] + '_norm.txt')
    scores3 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    score_path=os.path.join('Scores/LA', Database, Models[4] + '_norm.txt')
    scores4 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    p1,p2,p3,p4,p5=combinacion
    modelo_fusion[1] = scores0[1].apply(
        lambda x: weighted(x, p1)) + scores1[1].apply(
        lambda x: weighted(x, p2)) + scores2[1].apply(
        lambda x: weighted(x, p3)) + scores3[1].apply(
        lambda x: weighted(x, p4)) + scores4[1].apply(
        lambda x: weighted(x, p5))
    score_file_mf = os.path.join('Scores/LA', Database, '_Modelo_Fusion.txt')
    modelo_fusion.to_csv(path_or_buf=score_file_mf, sep=' ', header=False, index=False)
    eer, th = extract_eer(score_file = score_file_mf, metadata=os.path.join('Metadata', Database, 'trial_metadata.txt'), col_labels=5, phase='progress', pri=False)
    if eer<mejor_err:
        mejor_err=eer
        score_file_mff = os.path.join('Scores/LA', Database, 'GS_Ensemble_5.txt')
        modelo_fusion.to_csv(path_or_buf=score_file_mff, sep=' ', header=False, index=False)
        mejor_combinacion=combinacion
        print('Nueva mejor combinación:')
        print(combinacion)
        print('Mejor EER de progress: {}'.format(eer) )

'''

Nueva mejor combinación:
(0.1, 0.1, 0.1, 0.1, 0.6)
Mejor EER de progress: 0.055503623850338565
Nueva mejor combinación:
(0.1, 0.1, 0.15, 0.1, 0.55)
Mejor EER de progress: 0.054799560115503576
Nueva mejor combinación:
(0.1, 0.1, 0.25, 0.1, 0.45)
Mejor EER de progress: 0.05429836357028178
Nueva mejor combinación:
(0.1, 0.1, 0.3, 0.1, 0.4)
Mejor EER de progress: 0.052625717943272074
Nueva mejor combinación:
(0.1, 0.1, 0.35, 0.1, 0.35)
Mejor EER de progress: 0.052056899001512544
Nueva mejor combinación:
(0.1, 0.1, 0.4, 0.1, 0.3)
Mejor EER de progress: 0.05185403181189935
Nueva mejor combinación:
(0.1, 0.1, 0.45, 0.1, 0.25)
Mejor EER de progress: 0.05075020512664916
Nueva mejor combinación:
(0.1, 0.1, 0.5, 0.1, 0.2)
Mejor EER de progress: 0.04957875604486125
Nueva mejor combinación:
(0.1, 0.1, 0.5, 0.15, 0.15)
Mejor EER de progress: 0.04894231470656399
Nueva mejor combinación:
(0.1, 0.1, 0.55, 0.1, 0.15)
Mejor EER de progress: 0.04529272266812477
Nueva mejor combinación:
(0.1, 0.1, 0.6, 0.1

In [59]:
Database = 'ASVspoof2021_LA_eval'
score_file_mff = os.path.join('Scores/LA', Database, 'GS_Ensemble_5.txt')
eer, th = extract_eer(score_file = score_file_mff, metadata=os.path.join('Metadata', Database, 'trial_metadata.txt'), col_labels=5, phase='eval')

EER: 5.2095. Threshold: 0.68692



### DF

In [84]:
'''
Models=['CQCC_GMM', 'LFCC_GMM', 'LFCC_LCNN', 'RawNet2']
import itertools
pesos = [0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.7]
combinaciones = itertools.product(pesos, repeat=4)
# Filtrar las combinaciones que sumen 1
combinaciones_validas = filter(lambda x: sum(x) == 1, combinaciones)

# Iterar sobre las combinaciones válidas
Database = 'ASVspoof2021_DF_eval'
score_path=os.path.join('Scores/DF', Database, Models[0] + '_norm.txt')
scores = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
modelo_fusion=scores
mejor_err=100
for combinacion in combinaciones_validas:
    score_path=os.path.join('Scores/DF', Database, Models[0] + '_norm.txt')
    scores0 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    score_path=os.path.join('Scores/DF', Database, Models[1] + '_norm.txt')
    scores1 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    score_path=os.path.join('Scores/DF', Database, Models[2] + '_norm.txt')
    scores2 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    score_path=os.path.join('Scores/DF', Database, Models[3] + '_norm.txt')
    scores3 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    p1,p2,p3,p4=combinacion
    modelo_fusion[1] = scores0[1].apply(
        lambda x: weighted(x, p1)) + scores1[1].apply(
        lambda x: weighted(x, p2)) + scores2[1].apply(
        lambda x: weighted(x, p3)) + scores3[1].apply(
        lambda x: weighted(x, p4))
    score_file_mf = os.path.join('Scores/DF', Database, '_Modelo_Fusion.txt')
    modelo_fusion.to_csv(path_or_buf=score_file_mf, sep=' ', header=False, index=False)
    eer, th = extract_eer(score_file = score_file_mf, metadata=os.path.join('Metadata', Database, 'trial_metadata.txt'), col_labels=5, phase='progress', pri=False)
    if eer<mejor_err:
        mejor_err=eer
        score_file_mff = os.path.join('Scores/DF', Database, 'GS_Ensemble.txt')
        modelo_fusion.to_csv(path_or_buf=score_file_mff, sep=' ', header=False, index=False)
        mejor_combinacion=combinacion
        print('Nueva mejor combinación:')
        print(combinacion)
        print('Mejor EER de progress: {}'.format(eer) )
    
'''

Nueva mejor combinación:
(0.1, 0.1, 0.1, 0.7)
Mejor EER de progress: 0.07056102870891026


In [60]:
Database = 'ASVspoof2021_DF_eval'
score_file_mff = os.path.join('Scores/DF', Database, 'GS_Ensemble.txt')
eer, th = extract_eer(score_file = score_file_mff, metadata=os.path.join('Metadata', Database, 'trial_metadata.txt'), col_labels=5, phase='eval')

EER: 23.6331. Threshold: 0.58832



#### 4 baseline models + clasification model

In [17]:
'''
Models=['CQCC_GMM', 'LFCC_GMM', 'LFCC_LCNN', 'RawNet2', 'ModeloClasificacion']
import itertools
pesos = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]
combinaciones = itertools.product(pesos, repeat=5)
# Filtrar las combinaciones que sumen 1
combinaciones_validas = filter(lambda x: sum(x) == 1, combinaciones)

# Iterar sobre las combinaciones válidas
Database = 'ASVspoof2021_DF_eval'
score_path=os.path.join('Scores/DF', Database, Models[0] + '_norm.txt')
scores = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
modelo_fusion=scores
mejor_err=100
for combinacion in combinaciones_validas:
    score_path=os.path.join('Scores/DF', Database, Models[0] + '_norm.txt')
    scores0 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    score_path=os.path.join('Scores/DF', Database, Models[1] + '_norm.txt')
    scores1 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    score_path=os.path.join('Scores/DF', Database, Models[2] + '_norm.txt')
    scores2 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    score_path=os.path.join('Scores/DF', Database, Models[3] + '_norm.txt')
    scores3 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    score_path=os.path.join('Scores/DF', Database, Models[4] + '_norm.txt')
    scores4 = pd.read_csv(score_path, sep=" ", header=None, skipinitialspace=True)
    p1,p2,p3,p4,p5=combinacion
    modelo_fusion[1] = scores0[1].apply(
        lambda x: weighted(x, p1)) + scores1[1].apply(
        lambda x: weighted(x, p2)) + scores2[1].apply(
        lambda x: weighted(x, p3)) + scores3[1].apply(
        lambda x: weighted(x, p4)) + scores4[1].apply(
        lambda x: weighted(x, p5))
    score_file_mf = os.path.join('Scores/DF', Database, '_Modelo_Fusion.txt')
    modelo_fusion.to_csv(path_or_buf=score_file_mf, sep=' ', header=False, index=False)
    eer, th = extract_eer(score_file = score_file_mf, metadata=os.path.join('Metadata', Database, 'trial_metadata.txt'), col_labels=5, phase='progress', pri=False)
    if eer<mejor_err:
        mejor_err=eer
        score_file_mff = os.path.join('Scores/DF', Database, 'GS_Ensemble_5.txt')
        modelo_fusion.to_csv(path_or_buf=score_file_mff, sep=' ', header=False, index=False)
        mejor_combinacion=combinacion
        print('Nueva mejor combinación:')
        print(combinacion)
        print('Mejor EER de progress: {}'.format(eer) )

'''

Nueva mejor combinación:
(0.1, 0.1, 0.1, 0.1, 0.6)
Mejor EER de progress: 0.024641947577492523
Nueva mejor combinación:
(0.1, 0.1, 0.1, 0.15, 0.55)
Mejor EER de progress: 0.01993947878052437
Nueva mejor combinación:
(0.1, 0.1, 0.1, 0.2, 0.5)
Mejor EER de progress: 0.01735085439322337
Nueva mejor combinación:
(0.1, 0.1, 0.1, 0.25, 0.45)
Mejor EER de progress: 0.014734222462557359
Nueva mejor combinación:
(0.1, 0.1, 0.1, 0.3, 0.4)
Mejor EER de progress: 0.011273387431701021
Nueva mejor combinación:
(0.1, 0.1, 0.1, 0.35, 0.35)
Mejor EER de progress: 0.010571884577741417


In [61]:
Database = 'ASVspoof2021_DF_eval'
score_file_mff = os.path.join('Scores/DF', Database, 'GS_Ensemble_5.txt')
eer, th = extract_eer(score_file = score_file_mff, metadata=os.path.join('Metadata', Database, 'trial_metadata.txt'), col_labels=5, phase='eval')

EER: 11.3928. Threshold: 0.60410

